## Templeton metadata

In [5]:
#Import packages
import pandas as pd
import h5py
import numpy as np
import scipy as sp
import scipy.signal as sg
import xarray as xr
import os
import matplotlib.pyplot as plt
from matplotlib import patches
import ast

from DR_analysis_utils import Session, makePSTH, make_neuron_time_trials_tensor

%matplotlib notebook

In [6]:
#Set paths to experiment folders
main_path = [
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-07-26_14-09-36_620263\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-07-27_13-57-17_620263\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-08-02_15-40-19_620264\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-09-19_13-48-26_628801\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-09-26_12-48-09_636397\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-09-27_11-37-08_636397\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-12-05_13-08-02_644547\processed",
    r"\\allen\programs\mindscope\workgroups\templeton\TTOC\pilot recordings\2022-12-06_12-35-35_644547\processed",
]

### Loop through sessions to aggregate metadata

In [56]:
summary_metadata={}

for ip,path in enumerate(main_path):
    session=Session(path=path)
    if ip==0:
        summary_metadata['path']=[]
    summary_metadata['path'].append(path)
    
    # N trials per stimulus
    for stim in session.trials['trialStimID'].unique():
        stim_trials=len(session.trials.query('trialStimID == @stim'))
        if ip==0:
            summary_metadata[stim+'_trials']=[]
        summary_metadata[stim+'_trials'].append(stim_trials)
        
    # hit rate, relevant and irrelevant false alarm
    if ip == 0:
        summary_metadata['stim_rewarded']=[]
        summary_metadata['hit_rate']=[]
        summary_metadata['catch_rate']=[]
        summary_metadata['fa_rate_relevant']=[]
        summary_metadata['fa_rate_irrelevant']=[]
        
    summary_metadata['stim_rewarded'].append(session.trials['trialstimRewarded'][0])
    
    go_trials=session.trials.query('trialStimID == trialstimRewarded')
    hit_rate=go_trials['trial_response'].mean()
    summary_metadata['hit_rate'].append(hit_rate)
    
    catch_trials=session.trials.query('trialStimID == "catch"')
    catch_rate=catch_trials['trial_response'].mean()
    summary_metadata['catch_rate'].append(catch_rate)
    
    cross_modal=np.zeros(len(session.trials)).astype(bool)
    intra_modal=np.zeros(len(session.trials)).astype(bool)

    for (tt,trial) in session.trials.iterrows():
        if trial['trialStimID']=='catch':
            continue
        elif trial['trialStimID'][:3]==trial['trialstimRewarded'][:3]:
            intra_modal[tt]=True
            cross_modal[tt]=False
        else:
            intra_modal[tt]=False
            cross_modal[tt]=True
            
    fa_rate_intra=session.trials['trial_response'][(intra_modal)&
                                                   (session.trials['trialStimID']!=
                                                    session.trials['trialstimRewarded']).values].mean()
    summary_metadata['fa_rate_relevant'].append(fa_rate_intra)
    
    fa_rate_cross=session.trials['trial_response'][cross_modal].mean()
    summary_metadata['fa_rate_irrelevant'].append(fa_rate_cross)


    # N probes
    if ip == 0:
        summary_metadata['n_probes']=[]
    summary_metadata['n_probes'].append(len(session.good_units['probe'].unique()))
    
    
    # N good units per probe
    for pr in ['A','B','C','D','E','F']:
        if ip==0:
            summary_metadata['probe'+pr+'_units']=[]
        pr_units=len(session.good_units.query('probe == @pr'))
        summary_metadata['probe'+pr+'_units'].append(pr_units)
    
    
# N responsive units to each stimulus (or proportion) ... do this later


summary_metadata = pd.DataFrame.from_dict(summary_metadata)

In [57]:
summary_metadata

,path,vis1_trials,vis2_trials,sound2_trials,sound1_trials,catch_trials,stim_rewarded,hit_rate,catch_rate,fa_rate_relevant,fa_rate_irrelevant,n_probes,probeA_units,probeB_units,probeC_units,probeD_units,probeE_units,probeF_units
0,\\allen\programs\mindscope\workgroups\templeto...,158,150,154,153,58,vis1,0.993671,0.051724,0.086667,0.058632,3,0,156,174,0,0,207
1,\\allen\programs\mindscope\workgroups\templeto...,157,151,155,152,77,vis1,0.987261,0.000000,0.059603,0.026059,4,26,103,72,0,0,212
2,\\allen\programs\mindscope\workgroups\templeto...,156,156,155,160,75,sound1,1.000000,0.026667,0.012903,0.006410,4,52,89,91,0,0,220
3,\\allen\programs\mindscope\workgroups\templeto...,104,97,99,100,54,vis1,0.980769,0.407407,0.556701,0.462312,4,204,134,146,0,0,308
4,\\allen\programs\mindscope\workgroups\templeto...,140,140,140,145,64,sound1,0.917241,0.171875,0.235714,0.089286,4,122,82,46,0,0,130
5,\\allen\programs\mindscope\workgroups\templeto...,115,112,112,116,41,sound1,0.905172,0.292683,0.383929,0.259912,4,89,41,166,0,0,220
6,\\allen\programs\mindscope\workgroups\templeto...,164,160,160,160,57,vis1,0.737805,0.017544,0.006250,0.018750,4,155,125,181,0,0,244
7,\\allen\programs\mindscope\workgroups\templeto...,154,150,145,147,82,vis1,0.980519,0.134146,0.033333,0.075342,4,79,156,195,0,0,94
